# Parse PDF File for table

In [ ]:
# Install necessary libraries 
!pip install pdfplumber

In [3]:
import pdfplumber

pdf_path = "..\..\public\client_files\collin-college\collin.pdf"  # Insert the path to the PDF file
csv_file_path = "..\..\public\client_files\collin-college\collin_college_csv"  # Insert the path to the CSV file

with pdfplumber.open(pdf_path) as pdf:
    extracted_text = ""
    tables = []

    for page_num, page in enumerate(pdf.pages):
        extracted_text += page.extract_text() + "\n\n"
        tables_on_page = page.extract_tables()

        if tables_on_page:
            for table in tables_on_page:
                tables.append(table)

In [ ]:
for i, table in enumerate(tables):
    print(f"\nTable {i + 1}:")
    for row in table:
        print(row)

In [5]:
all_rows = []

for table in tables:
    for row in table:
        all_rows.append(row)

split_tables = []
current_table = []


for row in all_rows:
    if len(row) > 2 and row[2] == "Phone":
        if current_table:
            split_tables.append(current_table)
        current_table = [row]
    else:
        current_table.append(row)

if current_table:
    split_tables.append(current_table)

In [ ]:
for i, split_table in enumerate(split_tables):
    print(f"\nSplit Table {i + 1}:")
    for row in split_table:
        print(row)

In [ ]:
filtered_tables = []
extraneous_tables = []
for i, split_table in enumerate(split_tables):
    filtered_table = []
    extraneous_table = []
    for row in split_table:
        if any(cell not in [None, ""] for cell in row):
            if any(cell in [None, ""] for cell in row) or len(row) < 3:
                extraneous_table.append(row.copy())
            else:
                filtered_table.append(row.copy())

    print(f"\nSplit Table {i + 1} (after removing empty rows):")
    for row in filtered_table:
        print(row)
    print(f"\nExtra Table {i + 1} (need to augment the split table):")
    for row in extraneous_table:
        print(row)

    filtered_tables.append(filtered_table)
    extraneous_tables.append(extraneous_table)

In [ ]:
# Augmenting the extra to the split table
index = next(i for i, row in enumerate(filtered_tables[4]) if "National Autism" in row[0])
print(filtered_tables[4][index][1], "\n")
filtered_tables[4][index][1] += "\n" + [" ".join(row[1] for row in extraneous_tables[4])][0]
print(filtered_tables[4][index][1], "\n")

index = next(i for i, row in enumerate(filtered_tables[5]) if "Life Path" in row[0])
print(filtered_tables[5][index][1], "\n")
filtered_tables[5][index][1] += " " + [" ".join(extraneous_tables[5][i][0] for i in range(0, 4))][0]
print(filtered_tables[5][index][1], "\n")


filtered_tables[8].append(extraneous_tables[8][0])
print(filtered_tables[8], "\n")

index = next(i for i, row in enumerate(filtered_tables[9]) if "Medical City" in row[0])
print(filtered_tables[9][index][0], "\n")
filtered_tables[9][index][0] += "\n" + [" ".join(extraneous_tables[9][i][0] for i in range(0, 2))][0]
filtered_tables[9][index][0] += "\n" + extraneous_tables[9][2][0].replace("\n", "")
print(filtered_tables[9][index][0], "\n")

index = next(i for i, row in enumerate(filtered_tables[12]) if "Collin County Health Care Services" in row[0])
print(filtered_tables[12][index][0], "\n")
filtered_tables[12][index][0] += extraneous_tables[12][0][0]
print(filtered_tables[12][index][0], "\n")

index = next(i for i, row in enumerate(filtered_tables[12]) if "Women’s Veteran" in row[0])
print(filtered_tables[12][index][0], "\n")
filtered_tables[12][index][0] += extraneous_tables[12][1][0]
print(filtered_tables[12][index][0], "\n")

index = next(i for i, row in enumerate(filtered_tables[13]) if "Islamic" in row[0])
print(filtered_tables[13][index][1], "\n")
filtered_tables[13][index][1] += "\n" + ["\n".join(extraneous_tables[13][i][1] for i in range(0, 4))][0]
print(filtered_tables[13][index][1], "\n")

filtered_tables[13].append(extraneous_tables[13][4])
print(filtered_tables[13], "\n")

In [ ]:
filtered_table = [[
    [cell.replace('\n', ' ') if idx == 1 else cell for idx, cell in enumerate(row)] 
    for row in filtered_table
] for filtered_table in filtered_tables]

for i, test_table in enumerate(filtered_table):
    print(f"\nTest Table {i + 1}:")
    for row in test_table:
        print(row[1]) if len(row) > 1 else print(row)

In [10]:
from urllib.parse import urlparse
import re

def is_url(url):
    try:
        result = urlparse(url, scheme="https")
        result = urlparse(result.geturl().replace("///", "//"))
        return all([result.scheme, result.netloc])
    except AttributeError:
        return False

link_start_condition = lambda line: bool(line) and "www" in line or "http" in line or ".org" in line
link_possible_condition = lambda line: bool(line) and "/" in line or "-" in line or "aspx" in line or ".com" in line
link_condition = lambda link: bool(link) and is_url(link) \
    and link[-1] != "." and not '@' in link

email_condition = lambda line: bool(line) and "@" in line
email_end_condition = lambda line: bool(line) and len(line) > 3 and \
    line[-4] == "." and all(char.isalpha() for char in line[-3:]) or "org" in line

phone_regex = r"((?:\+\d{1}[-\.\s]??|\d{4}[-\.\s]??)?(?:\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\w{4}))"
phone_extra_condition = lambda line: bool(line) and line[0] == "(" and line[-1] == ")"

address_start_condition = lambda line: bool(line) and line[0].isdigit() and \
    len(line) > 5 and not "-" in line
address_condition = lambda address: bool(address) and address[0].isdigit() and \
    len(address) > 5 and all(char.isdigit() for char in address[-5:])

def process_first_column(row):
    lines = [line.strip() for line in row[0].split("\n")]

    name = ""
    line_iter = 0
    while lines:
        line = lines[0]
        if link_start_condition(line) or address_start_condition(line) \
            or email_condition(line) or line_iter >= 2:
            break
        name += " " + lines.pop(0)
        line_iter += 1

    links = []
    addresses = []
    others = []
    emails = []
    prev_link = ""
    prev_address = ""
    while lines:
        if email_condition(lines[0]):
            emails.append(lines.pop(0))
            continue
        line = lines[0]
        if address_condition(prev_address):
            addresses.append(prev_address)
            prev_address = ""
        if address_start_condition(line):
            if link_condition(prev_link):
                links.append(prev_link)
                prev_link = ""
            prev_address += line
        elif link_start_condition(line.lower()):
            prev_link = line
        elif bool(prev_address):
            prev_address += " " + line
        elif bool(prev_link) and link_possible_condition(line):
            prev_link += line
        else:
            if link_condition(prev_link):
                links.append(prev_link)
            elif bool(prev_link):
                others.append(prev_link)
            prev_link = ""
            if address_condition(prev_address):
                addresses.append(prev_address)
            elif bool(prev_address):
                others.append(prev_address)
            prev_address = ""
            others.append(line)
        lines.pop(0)
    links.append(prev_link) if bool(prev_link) else None
    addresses.append(prev_address) if bool(prev_address) else None

    return (
        [link.replace(" ", "") for link in links],
        addresses,
        emails,
        name.strip(),
        others,
    )

def process_second_column(row):
    return row[1].replace('\n', ' ').strip()

def process_third_column(row):
    lines = [line.strip() for line in row[2].split("\n")]

    emails = []
    phones = []
    links = []
    others = []
    prev_email = ""
    prev_link = ""
    prev_descriptor = ""

    while lines:
        possible_phone = re.findall(phone_regex, lines[0])
        if possible_phone:
            phone = possible_phone[0]
            lines.pop(0)
            if(lines and phone_extra_condition(lines[0])):
                phone += " " + lines.pop(0)
            if(bool(prev_descriptor)):
                phones.append(phone + " [" + prev_descriptor + "]")
            elif(lines and len(lines[0]) < 9):
                phones.append(phone + " [" + lines.pop(0) + "]")
            else:
                phones.append(phone)
            continue
        if bool(prev_descriptor):
            others.append(prev_descriptor)
            prev_descriptor = ""
        line = lines[0]
        if bool(prev_email) and email_end_condition(line):
            prev_email += line
        elif bool(prev_link) and link_possible_condition(line):
            prev_link += line
        elif email_condition(line):
            prev_email = line
        elif link_start_condition(line.lower()):
            prev_link = line
        else:
            if link_condition(prev_link):
                links.append(prev_link)
            elif bool(prev_link):
                others.append(prev_link)
            if email_condition(prev_email):
                emails.append(prev_email)
            elif bool(prev_email):
                others.append(prev_email)
            prev_link = ""
            prev_email = ""
            prev_descriptor = line
        lines.pop(0)
    links.append(prev_link) if bool(prev_link) else None
    others.append(prev_descriptor) if bool(prev_descriptor) else None
    emails.append(prev_email) if bool(prev_email) else None

    return (
        [link.replace(" ", "") for link in links],
        emails,
        phones,
        others,
    )

In [ ]:
resources = []
types = [
    "Community Resources",
    "Adult Shelters",
    "Youth Only Shelters",
    "Transitional Living Programs",
    "Support Groups",
    "Counseling Services",
    "Domestic Violence Services",
    "Drug Treatment Referrals",
    "Gay, Lesbian, Bisexual and Transgender",
    "Psychiatric Hospitals",
    "Vocation / Employment",
    "Hotlines",
    "Medical Referrals",
    "Other Miscellaneous Referrals",
]

for i, test_table in enumerate(filtered_table):
    test_table = test_table[1:]
    for row in test_table:
        links, addresses, emails, name, others = process_first_column(row)
        description = process_second_column(row)
        links2, emails2, phones, others2 = process_third_column(row)
        resources.append(
            {
                "name": name,
                "links": links + links2,
                "addresses": addresses,
                "emails": emails + emails2,
                "description": description,
                "phone": phones,
                "type": [types[i]],
                "others": others + others2,
            }
        )
for resource in resources:
    print(resource)

In [15]:
# Write to CSV
import csv

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Links", "Addresses", "Emails", "Phone", "Type", "Description"])
    for resource in resources:
        writer.writerow([
            resource["name"],
            ", ".join(resource["links"]),
            ", ".join(resource["addresses"]),
            ", ".join(resource["emails"]),
            ", ".join(resource["phone"]),
            ", ".join(resource["type"]),
            resource["description"],
        ])